# Natural Language Processing

NLP is a discipline in computing that delas with teh communication between natural (human) languages and computer languages. A common example of NLP is something like spellcheck or autocomplete. Essentially NLP is the field that focuses on how computers can understand and/or process natural/human languages.

## Recurrent Neural Networks

A neural network that is much more capable of processing sequential data such as text or characters called a recurrent neural network (RNN).

*   Sentiment Analysis
*   Character Generation

RNN's are fairly complex and come in many different forms.

### Bag of words

In [0]:
vocab = {}
word_encoding = 1

def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ") # Create a list of all of the words in the text.
  bag = {} # Stores all encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word] # get encoding from vocab

    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "This is a test to see if this test will work is is test a a"

bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


### Integer Encoding

This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words.

In [0]:
vocab = {}
word_encoding = 1

def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


### Word Embeddings

This method keeps the order of words intact as well as encodes similar words with very similar labels. It attemtps to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.

## Recurrent Neural Networks (RNN's)

We human don't process text all at once. We read word by word from left to right and keep track of the current meaning of the sentence so we can understand the meaning of the next word. Well this is exactly what a recurrent neural network is designed to do. When we say recurrent neural network all we really mean is a network that contains a loop. A RNN will process one word at a time while maintaining an internal memory of what it's already seen. This will allow it to treat words differently based on their order in a sentence and to slowly build an understanding of the entire input, one word at a time.

### LSTM

The layer we dicussed in depth above was called a *simpleRNN*. However, there does exist some other recurrent layers (layers that contain a loop) that work much better than a simple RNN layer. The one we will talk about here is called LSTM (Long Short-Term Memory). This layer works very similarily to the simpleRNN layer but adds a way to access inputs from any timestep in the past. Whereas in our simple RNN layer input from previous timestamps gradually disappeared as we got further through the input. With a LSTM we have a long-term memory data structure storing all the previously seen inputs as well as when we saw them. This allows for us to access any previous value we want at any point in time. This adds to the complexity of our network and allows it to discover more useful relationships between inputs and when they appear. 

For the purpose of this course we will refrain from going any further into the math or details behind how these layers work.

### Sentimental Analysis


In [1]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np
keras = tf.keras

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

TensorFlow 2.x selected.


Using TensorFlow backend.


17465344/17464789 [==============================] - 1s 0us/step


#### More Preprocessing

If we have a look at some of our loaded in reviews we'll notice that they are different lengths. This is an issue. We cannot pass different length data into our neural network. Therefore we must make each review the same length. 

In [0]:
# If the review is greater than 250 words then trim off the extra words
# If the review is less than 250 words add the necessary amount of 0's to make it equal to 250

train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data  = sequence.pad_sequences(test_data,  MAXLEN)

#### Creating the model


In [0]:
model = tf.keras.Sequential([
                    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
                    tf.keras.layers.LSTM(32),
                    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


### Training

In [0]:
model.compile(loss='binary_crossentropy', optimizer ='rmsprop', metrics = ['acc'])

history = model.fit(train_data, train_labels, epochs = 10, validation_split = 0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 23s 1ms/sample - loss: 0.4155 - acc: 0.8123 - val_loss: 0.3065 - val_acc: 0.8752
Epoch 2/10
20000/20000 [==============================] - 16s 779us/sample - loss: 0.2392 - acc: 0.9092 - val_loss: 0.2709 - val_acc: 0.8898
Epoch 3/10
20000/20000 [==============================] - 15s 746us/sample - loss: 0.1809 - acc: 0.9319 - val_loss: 0.3612 - val_acc: 0.8862
Epoch 4/10
20000/20000 [==============================] - 15s 742us/sample - loss: 0.1570 - acc: 0.9440 - val_loss: 0.2992 - val_acc: 0.8750
Epoch 5/10
20000/20000 [==============================] - 15s 766us/sample - loss: 0.1297 - acc: 0.9544 - val_loss: 0.3361 - val_acc: 0.8886
Epoch 6/10
20000/20000 [==============================] - 15s 746us/sample - loss: 0.1102 - acc: 0.9618 - val_loss: 0.3199 - val_acc: 0.8900
Epoch 7/10
20000/20000 [==============================] - 15s 736us/sample - loss: 0.0942 - acc: 0.9677 - v

In [0]:
results = model.evaluate(test_data, test_labels)
print(results)

25000/25000 [==============================] - 4s 178us/sample - loss: 0.4431 - acc: 0.8538
[0.4431117473435402, 0.8538]


#### Making Predictions


In [0]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]


text = 'that movie was amazing, so amazing'
encoded = encode_text(text)
print(encoded)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  12  17  1

In [0]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was amazing so amazing


In [0]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)


[0.9405474]
[0.30330217]


## RNN Play Generator

We will simply show the RNN an example of something we want it to recreate and it will learn how to write a version of it on its own. We'll do this using a character predicitve model that will take as input a variable length sequence and predict the next character. We can use the model many times in a row with the output from the last prediction as the input for the next call to generate a sequence.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
from google.colab import files:
  path_to_file = list(files.upload().keys())[0] ## make sure it is text file

SyntaxError: ignored

In [5]:
# Read, then decode for py2 compat
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of textt: {} characters'.format(len(text)))

Length of textt: 1115394 characters


### Encoding

In [6]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

# Let's look at how part of our text is encoded
print("Text: ", text[:13])
print("Encoded: ", text_to_int(text[:13]))


def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]
First Citizen


### Creating Training Examples


In [0]:
seq_length = 100 # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

## Batch method to turn this stream of characters into batches of desired length.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

## Sequence of length 101 and split them into input and output
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text


dataset = sequences.map(split_input_target)


In [0]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements.)
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

### Building a model

In [14]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
                               tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                                         batch_input_shape=[batch_size, None]),
                               tf.keras.layers.LSTM(rnn_units,
                                                    return_sequences=True,
                                                    stateful=True,
                                                    recurrent_initializer='glorot_uniform'),
                               tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


### Creating a loss function

In [16]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # ask our model for a prediction on our first batch of training data
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [0]:
## We can see that the prediction is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

In [0]:
# lets examine one prediction

pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each step

In [0]:
# and finally will look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values represnting the probabilities of each character occuring next

In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

";?Pcc:FbUOYFKn'vpBr3G IRH.JQamvlbRXXYHU\nl&BGvLtQpMonOwG .GrfePcm$n EfJr'-ydPzlIQ-gRhijj\nymmYGi'uBCAE"

In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

Train for 172 steps
Epoch 1/50
172/172 [==============================] - 12s 68ms/step - loss: 2.5568
Epoch 2/50
172/172 [==============================] - 11s 62ms/step - loss: 1.8623
Epoch 3/50
172/172 [==============================] - 11s 62ms/step - loss: 1.6187
Epoch 4/50
172/172 [==============================] - 11s 63ms/step - loss: 1.4887
Epoch 5/50
172/172 [==============================] - 11s 62ms/step - loss: 1.4105
Epoch 6/50
172/172 [==============================] - 11s 62ms/step - loss: 1.3551
Epoch 7/50
172/172 [==============================] - 11s 63ms/step - loss: 1.3104
Epoch 8/50
172/172 [==============================] - 11s 62ms/step - loss: 1.2701
Epoch 9/50
172/172 [==============================] - 11s 62ms/step - loss: 1.2335
Epoch 10/50
172/172 [==============================] - 11s 62ms/step - loss: 1.1972
Epoch 11/50
172/172 [==============================] - 11s 62ms/step - loss: 1.1609
Epoch 12/50
172/172 [==============================] - 11s 62ms/s

AttributeError: ignored

In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))